In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import csv

def assign_crimes_no_sum(crime_df, hex_grid, date):
    
    #convert dataframe to GeoDataFrame
    crime_df['FIRST_OCCURRENCE_DATE'] = pd.to_datetime(crime_df['FIRST_OCCURRENCE_DATE'], errors='coerce')
    crime_df['date'] = crime_df['FIRST_OCCURRENCE_DATE'].dt.date
    daily_crime_df = crime_df[crime_df['date'] == date]

    daily_crime_df = daily_crime_df.copy()
    daily_crime_df.loc[:, 'geometry'] = daily_crime_df.apply(lambda row: Point(row['GEO_LON'], row['GEO_LAT']), axis=1)
    crime_gdf = gpd.GeoDataFrame(daily_crime_df, geometry='geometry', crs="EPSG:4326")

    #ensure both GeoDataFrames have the same CRS
    hex_grid = hex_grid.to_crs(epsg=4326)

    #Assign crimes to a hexagon
    crimes_with_hex = gpd.sjoin(crime_gdf, hex_grid, how="left", predicate="within")

    return crimes_with_hex

crime_data = "val_data.csv" #.csv
crime_df = pd.read_csv(crime_data)
grid_file = "grid.geojson" #.geojson
hex_grid = gpd.read_file(grid_file)
save_name = "test.csv" #.csv

#get unique dates
crime_df['FIRST_OCCURRENCE_DATE'] = pd.to_datetime(crime_df['FIRST_OCCURRENCE_DATE'], errors='coerce')
crime_df['date'] = crime_df['FIRST_OCCURRENCE_DATE'].dt.date

unique_dates = sorted(crime_df['date'].dropna().unique())

#print(unique_dates[0:5])

crime_grids = []

#create grid of crime counts for each day
for date in unique_dates:
    
    grid = assign_crimes_no_sum(crime_df, hex_grid, date)
    crime_grids.append(grid)

crime_df = pd.concat(crime_grids, ignore_index=True)
crime_df.to_csv(save_name, index=False)